In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from tqdm import tqdm_notebook as tqdm
from sklearn.metrics import mean_squared_error, accuracy_score, classification_report
RMSE = lambda y_true,y_pred: np.sqrt(mean_squared_error(y_true, y_pred))

In [2]:
FEATURE_AGGREGATION = 'subtr'
TRAIN_TEST_PARTITION = 'shuffled'
LAMBDA = 10

results = {'lambda': LAMBDA}

In [3]:
# %store -r FEATURE_AGGREGATION
# %store -r TRAIN_TEST_PARTITION

In [4]:
data_tr = pd.read_csv(f"./data/{FEATURE_AGGREGATION}/{TRAIN_TEST_PARTITION}_train.csv")
data_v = pd.read_csv(f"./data/{FEATURE_AGGREGATION}/{TRAIN_TEST_PARTITION}_valid.csv")
data_ts = pd.read_csv(f"./data/{FEATURE_AGGREGATION}/test.csv").astype(np.int32)

In [5]:
Xtr = data_tr.drop('target', axis=1).values
Xv = data_v.drop('target', axis=1).values
Xts = data_ts.drop('target', axis=1).values

ytr = data_tr.target.values
yv = data_v.target.values
yts = data_ts.target.values

### Linear regression

In [6]:
import linear_model as lm
from sklearn.pipeline import Pipeline

In [7]:
use_RBFs = FEATURE_AGGREGATION == 'concat' #only using RBFs when having lots of features

In [8]:
if use_RBFs:
    linear = Pipeline([
        ('rbf', lm.RBF_transformer(15, cache=True)),
        ('lin_r', lm.LinRegression(n_features=16, metric='accuracy', lambda_=LAMBDA))
    ])

    logistic = Pipeline([
        ('rbf', lm.RBF_transformer(15, cache=True)),
        ('log_r', lm.LogRegression(n_features=16, metric='accuracy', lambda_=LAMBDA))
    ])
else:
    linear = Pipeline([
        ('lin_r', lm.LinRegression(n_features=Xtr.shape[1], metric='accuracy', lambda_=LAMBDA))
    ])
    logistic = Pipeline([
        ('log_r', lm.LogRegression(n_features=Xtr.shape[1], metric='accuracy', lambda_=LAMBDA))
    ])

In [9]:
sampler = lm.Subsampler(1, y_oh=False, normalize=not use_RBFs, neg_weight=1)

In [10]:
sampler.fit(Xtr, ytr);

In [11]:
Xv_ss, yv_ss = sampler.transform(Xv, yv, sample=True)

In [12]:
linear.fit(Xtr, ytr,
           lin_r__batch_generator = sampler,
           lin_r__n_epochs=100,
           lin_r__valid_set=(Xv_ss, yv_ss));

HBox(children=(IntProgress(value=0, max=99), HTML(value='')))

In [13]:
logistic.fit(Xtr, ytr,
           	 log_r__batch_generator = sampler,
           	 log_r__n_epochs=100,
           	 log_r__valid_set=(Xv_ss, yv_ss));

HBox(children=(IntProgress(value=0, max=99), HTML(value='')))

In [14]:
results['linear_accuracy_ss'] = linear.score(Xv_ss, yv_ss)
results['linear_accuracy_full'] = linear.score(Xv,yv)

In [15]:
results['logistic_accuracy_ss'] = logistic.score(Xv_ss, yv_ss)
results['logistic_accuracy_full'] = logistic.score(Xv, yv)

In [16]:
lin_r = [x[1] for x in linear.steps if x[0] == 'lin_r'].pop()
log_r = [x[1] for x in logistic.steps if x[0] == 'log_r'].pop()

In [17]:
lin_r.metric = 'RMSE'
log_r.metric = 'RMSE'

In [18]:
results['linear_RMSE_ss'] = linear.score(Xv_ss, yv_ss)
results['linear_RMSE_full'] = linear.score(Xv, yv)

In [19]:
results['logistic_RMSE_ss'] = logistic.score(Xv_ss, yv_ss)
results['logistic_RMSE_full'] = logistic.score(Xv, yv)

---
### keras

In [20]:
from keras.models import Model
from keras.layers import Input, Dense, Dropout
from keras.optimizers import SGD

/home/wolterlw/miniconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [21]:
HIDDEN1_SIZE = 16
HIDDEN2_SIZE = 4

In [22]:
inp = Input((Xtr.shape[1], ))
x = Dense(HIDDEN1_SIZE, activation='relu')(inp)
x = Dense(HIDDEN2_SIZE, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(2, activation='softmax')(x)

model = Model(inputs=[inp], outputs=[x])

In [23]:
model.compile(optimizer='SGD', loss='categorical_crossentropy', metrics=['accuracy'])

In [24]:
sampler_nn = lm.Subsampler(5000, y_oh=True, normalize=True, neg_weight=3)

In [25]:
sampler_nn.fit(Xtr, ytr)
sampler_nn.neg_weight = 1
Xv_ss, yv_ss = sampler_nn.transform(Xv, yv, sample=True)
sampler_nn.neg_weight = 1

In [26]:
model.fit_generator(sampler_nn.get_generator(),
                    steps_per_epoch=3000,
                    epochs=20,
                    validation_data=(Xv_ss,yv_ss),
                    workers=3)

Epoch 1/20
3000/3000 [==============================] - 29s 10ms/step - loss: 0.6925 - acc: 0.5139 - val_loss: 0.6905 - val_acc: 0.5455
Epoch 2/20
3000/3000 [==============================] - 29s 10ms/step - loss: 0.6902 - acc: 0.5350 - val_loss: 0.6894 - val_acc: 0.5227
Epoch 3/20
3000/3000 [==============================] - 29s 10ms/step - loss: 0.6890 - acc: 0.5408 - val_loss: 0.6888 - val_acc: 0.5422
Epoch 4/20
3000/3000 [==============================] - 29s 10ms/step - loss: 0.6880 - acc: 0.5437 - val_loss: 0.6889 - val_acc: 0.5325
Epoch 5/20
3000/3000 [==============================] - 29s 10ms/step - loss: 0.6870 - acc: 0.5501 - val_loss: 0.6896 - val_acc: 0.5260
Epoch 6/20
3000/3000 [==============================] - 29s 10ms/step - loss: 0.6858 - acc: 0.5517 - val_loss: 0.6904 - val_acc: 0.5244
Epoch 7/20
3000/3000 [==============================] - 29s 10ms/step - loss: 0.6842 - acc: 0.5605 - val_loss: 0.6905 - val_acc: 0.5325
Epoch 8/20
3000/3000 [==========================

In [28]:
pred_ss = model.predict(Xv_s)

NameError: name 'Xv_s' is not defined